In [5]:
# 📒 Day 2 – Bond Basics & PV Calculation

# Imports
import numpy as np
import matplotlib.pyplot as plt , yfinance as yf
import pandas as pd
from datetime import datetime
from math import exp, log
from scipy.integrate import quad
import sympy as sp

In [49]:
# Function: Present Value of single cash flow
def present_value(cash_flow, rate, periods):
    return cash_flow / ((1 + rate) ** periods)

# Example
CF = 100  # Cash flow
r = 0.05  # 5% interest rate
n = 1     # in 1 year

pv = present_value(CF, r, n)
print(f"Present Value: {pv:.2f}")

Present Value: 95.24


In [52]:
# playing with functions 

def bond_return(face, price, coupon, years):
    total_interest = coupon * years
    capital_gain = face - price
    total_return = total_interest + capital_gain
    return_amt = total_return
    return_pct = total_return / price * 100
    print(f"Total Return Amount: ${return_amt:.2f}")
    print(f"Total Return %     : {return_pct:.2f}%")
    return return_amt

test = bond_return(1000, 970, 4, 2)
print(test)

Total Return Amount: $38.00
Total Return %     : 3.92%
38


In [8]:
# playing with loops

ytms = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]
for ytm in ytms:
    price = bond_return(face_value, market_price, annual_coupon_payment, maturity_years) / ((1 + ytm) ** maturity_years)
    print(f"When YTM is: {ytm*100:.1f}%, Price is: ${price:.2f}")

When YTM is: 1.0%, Price is: $266.17
When YTM is: 2.0%, Price is: $253.38
When YTM is: 3.0%, Price is: $241.31
When YTM is: 4.0%, Price is: $229.93
When YTM is: 5.0%, Price is: $219.19
When YTM is: 6.0%, Price is: $209.05
When YTM is: 7.0%, Price is: $199.46
When YTM is: 8.0%, Price is: $190.39
When YTM is: 9.0%, Price is: $181.82
When YTM is: 10.0%, Price is: $173.70


In [ ]:
# importing bonds data from yfinance

ticker = ['^IRX', '^FVX', '^TNX', '^TYX']  # 3M, 5Y, 10Y, 30Y Treasury
data = yf.download(ticker, start='2023-01-01', end=datetime.now().strftime('%Y-%m-%d'))['Adj Close']
data = data.dropna()  # Drop rows with NaN values
data.columns = ['3M', '5Y', '10Y', '30Y']  # Rename columns for clarity
data.head()  # Display the first few rows of the DataFrame

[*********************100%***********************]  1 of 4 completed

4 Failed downloads:
['^FVX', '^TNX', '^IRX', '^TYX']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


,3M,5Y,10Y,30Y
Date,,,,


[**********************75%***********            ]  3 of 4 completed

In [47]:
F = 100.0
T = 3.0
a, b = 0.28, 0.006           # r(s) = a + b*s


integral = a*T + 0.5*b*T**2  # ∫ r(s) ds
Z = np.exp(-integral)
price = F * Z
y = integral / T             # implied cont. yield

print(f"Discount factor Z = {Z:.6f}")
print(f"Price = {price:.2f}")
print(f"Implied cont. yield y = {y:.4%}")

Discount factor Z = 0.420210
Price = 42.02
Implied cont. yield y = 28.9000%


In [42]:
# Function: Discount factor for 0 coupon bond

def discount_factor(r_expr, T, face_value):
    integral = sp.integrate(r_expr, (s, 0, T))
    Z = sp.exp(-integral)
    P = face_value * Z
    y = integral / T
    print(f"Discount factor Z = {Z:.6f}")
    print(f"Price = {P:.2f}")   
    if hasattr(y, "evalf"):
        y = float(y.evalf())
    print(f"Implied cont. yield y = {y:.4%}")
    return Z, P, y

s = sp.symbols('s')

In [43]:
#case 1: r(s) is constant
r_const = 0.05
print('When r is constant and equals', r_const ,discount_factor(r_const, 3, 100))
print('---')

#case 2: r(s) is linear
r_linear = 0.028 + 0.006 * s
print(discount_factor(r_linear, 4, 100))
print('---')

#case 3: r(s) is sinusoidal
r_sin = 0.03 + 0.01 * sp.sin(sp.pi * s / 2)
print(discount_factor(r_sin, 2, 100))

Discount factor Z = 0.860708
Price = 86.07
Implied cont. yield y = 5.0000%
When r is constant and equals 0.05 (0.860707976425058, 86.0707976425058, 0.05000000000000001)
---
Discount factor Z = 0.852144
Price = 85.21
Implied cont. yield y = 4.0000%
(0.852143788966211, 85.2143788966211, 0.04)
---
Discount factor Z = 0.929850
Price = 92.98
Implied cont. yield y = 3.6366%
(0.941764533584249*exp(-0.04/pi), 94.1764533584249*exp(-0.04/pi), 0.03636619772367581)


In [46]:
def bondprice(coupon, periods, rate, face_value):
    pv_coupon = coupon * ((1 - (1/((1 + rate)**periods)))/rate) 
    pv_face = face_value/(1 + rate)**periods
    total_price = pv_coupon + pv_face
    
    print(f"Present Value of Coupons  : ${pv_coupon:,.2f}")
    print(f"Present Value of Face     : ${pv_face:,.2f}")
    print(f"Total Bond Price          : ${total_price:,.2f}")
    return total_price

print(bondprice(45, 40, 0.06, 1000))  # Coupon $45, 20 years, 6% yield, $1000 face value

Present Value of Coupons  : $677.08
Present Value of Face     : $97.22
Total Bond Price          : $774.31
774.3055469271264
